In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [14]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff
using SpecialFunctions
using Polynomials
using BenchmarkTools
using TransportMap
using QuadGK
using FastGaussQuadrature
using AdaptiveTransportMap: derivative, vander, scale!, evaluate
using DiffResults
using Distributions
using Random
using LoopVectorization
using Optim

In [33]:
B = BasisProHermite(5)
B[1](-0.6) - exp(-0.6^2/4)

0.0

In [34]:
Nx = 1
Ne = 500
ens = EnsembleState(Nx, Ne)
ens.S .= randn(Nx, Ne)


B = MultiBasis(BasisProHermite(3; scaled =true), Nx)

idx = reshape([0], (1, 1))
truncidx = idx[1:2:end,:]

coeff =  randn(size(idx,1))

f = ExpandedFunction(B, idx, coeff)

ExpandedFunction{4,1,1}(MultiBasis{4,1}(Basis of 4 functions:
ProHermite{0}(ProPolyHermite{0}(ImmutablePolynomial(0.6316187777460647), true), true)
ProHermite{1}(ProPolyHermite{1}(ImmutablePolynomial(0.6316187777460647*x), true), true)
ProHermite{2}(ProPolyHermite{2}(ImmutablePolynomial(-0.44662192086900115 + 0.44662192086900115*x^2), true), true)
ProHermite{3}(ProPolyHermite{3}(ImmutablePolynomial(-0.7735718587191167*x + 0.25785728623970555*x^3), true), true)
), [0], [-0.44955070418036275])

In [35]:
ψt_basis, dψt_basis, d2ψt_basis = alleval(f, ens)

([1.0; 1.0; … ; 1.0; 1.0], [0.0; 0.0; … ; 0.0; 0.0], [0.0; 0.0; … ; 0.0; 0.0])

In [48]:
@code_warntype f([0.5])

Variables
  f::ExpandedFunction{4,1,1}
  x::Array{Float64,1}
  out::Any
  @_4::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64
  fi::MultiFunction{4,1}

Body::Any
1 ─       (out = 0.0)
│   %2  = (1:$(Expr(:static_parameter, 2)))::Core.Compiler.Const(1:1, false)
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4::Core.Compiler.Const((1, 1), false) === nothing)::Core.Compiler.Const(false, false)
│   %5  = Base.not_int(%4)::Core.Compiler.Const(true, false)
└──       goto #4 if not %5
2 ─ %7  = @_4::Core.Compiler.Const((1, 1), false)::Core.Compiler.Const((1, 1), false)
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Core.Compiler.Const(1, false)
│   %10 = Base.getproperty(f, :B)::MultiBasis{4,1}
│   %11 = Base.getproperty(f, :idx)::Array{Int64,2}
│   %12 = i::Core.Compiler.Const(1, false)::Core.Compiler.Const(1, false)
│   %13 = Base.getindex(%11, %12, AdaptiveTransportMap.:(:))::Array{Int64,1}
│         (fi = AdaptiveTransportMap.MultiFunction(%10, %13))
│   %15 = out::C

In [ ]:
-f.coeff[1]*B.B[1](0.5)

In [47]:
@code_warntype ψ_basis = evaluate_basis(f, ens.S)
# dψ_basis = grad_x_basis(f, ens.S)
# d2ψ_basis = hess_x_basis(f, ens.S)

ErrorException: expression is not a function call, or is too complex for @code_warntype to analyze; break it down to simpler parts if possible

In [27]:
@testset "Test evaluation, gradient and hessian of expanded function Nx = 1 I" begin

    Nx = 1
    Ne = 500
    ens = EnsembleState(Nx, Ne)
    ens.S .= randn(Nx, Ne)


    B = MultiBasis(BasisProHermite(3; scaled =true), Nx)

    idx = reshape([0], (1, 1))
    truncidx = idx[1:2:end,:]

    coeff =  randn(size(idx,1))

    f = ExpandedFunction(B, idx, coeff)



    ψt_basis, dψt_basis, d2ψt_basis = alleval(f, ens)

    ψ_basis = evaluate_basis(f, ens.S)
    dψ_basis = grad_x_basis(f, ens.S)
    d2ψ_basis = hess_x_basis(f, ens.S);
    @test norm(ψt_basis - ψ_basis)<1e-8
    @test norm(dψt_basis - dψ_basis)<1e-8
    @test norm(d2ψt_basis - d2ψ_basis)<1e-8


    # For truncated basis
    ψtrunc_basis = evaluate_basis(f, ens.S, truncidx)
    dψtrunc_basis = grad_x_basis(f, ens.S, truncidx)
    d2ψtrunc_basis = hess_x_basis(f, ens.S, truncidx);

    @test norm(ψ_basis[:,1:2:end] - ψtrunc_basis)<1e-8
    @test norm(dψ_basis[:,1:2:end,:] - dψtrunc_basis)<1e-8
    @test norm(d2ψt_basis[:,1:2:end,:,:] - d2ψtrunc_basis)<1e-8


    # Verify function evaluation
    @test norm(map(i->f(member(ens,i)),1:Ne) - evaluate(f, ens.S))<1e-8

    #  Verify gradient
    dψ = grad_x(f, ens.S)
    @test norm(hcat(map(i->ForwardDiff.gradient(f, member(ens,i)), 1:Ne)...)' - dψ)<1e-8

    # Verify hessian
    d2ψ = hess_x(f, ens.S)

    for i=1:Ne
        d2ψt = ForwardDiff.hessian(f, member(ens,i))
        @test norm(d2ψt - d2ψ[i,:,:])<1e-8
    end


    # Verify grad_xd
    dψxd = grad_xd(f, ens.S)

    for i=1:Ne
        dψxd_t = ForwardDiff.gradient(f, member(ens,i))[end]
        @test abs(dψxd[i] - dψxd_t)<1e-10
    end

    # Verify hess_xd
    d2ψxd = hess_xd(f, ens.S)

    for i=1:Ne
        d2ψxd_t = ForwardDiff.hessian(f, member(ens,i))[end,end]
        @test abs(d2ψxd[i] - d2ψxd_t)<1e-10
    end

    # Verify grad_coeff
    dψcoeff  = grad_coeff(f, ens.S)
    dψcoefftrunc  = grad_coeff(f, ens.S, collect(1:2:size(idx,1)))
    @test norm(dψcoeff[:,1:2:end] - dψcoefftrunc)<1e-10

    @test norm(dψcoeff - ψ_basis)<1e-10
    @test norm(dψcoefftrunc - ψtrunc_basis)<1e-10

    # Verify hess_coeff
    d2ψcoeff  = hess_coeff(f, ens.S)
    d2ψcoefftrunc  = hess_coeff(f, ens.S, collect(1:2:size(idx,1)))

    @test norm(d2ψcoeff)<1e-10

    # Verify grad_coeff_grad_xd
    dψcoeffxd = grad_coeff_grad_xd(f, ens.S)
    dψcoeffxdtrunc = grad_coeff_grad_xd(f, ens.S, collect(1:2:size(idx,1)))

    @test norm(dψcoeffxd[:,1:2:end] - dψcoeffxdtrunc)<1e-10
    @test norm(dψcoeffxd - grad_xk_basis(f, ens.S, 1, Nx))<1e-10
    @test norm(dψcoeffxdtrunc - grad_xk_basis(f, ens.S, 1, Nx, f.idx[collect(1:2:size(idx,1)),:]))<1e-10

    # Verify hess_coeff_grad_xd
    d2ψcoeffxd = hess_coeff_grad_xd(f, ens.S)
    d2ψcoeffxdtrunc = hess_coeff_grad_xd(f, ens.S, collect(1:2:size(idx,1)))

    @test norm(d2ψcoeffxd)<1e-10
    @test norm(d2ψcoeffxdtrunc)<1e-10
end

Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[27]:25
  Expression: norm(ψt_basis - ψ_basis) < 1.0e-8
   Evaluated: 11.306324020932555 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[27]:25
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[27]:26
  Expression: norm(dψt_basis - dψ_basis) < 1.0e-8
   Evaluated: 4.2122654069914205 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[27]:26
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[27]:27
  Expression: norm(d2ψt_basis - d2ψ_basis) < 1.0e-8
   Evaluated: 4.874835725340076 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[27]:27
 [2] to

   Evaluated: 0.1861228117853838 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[27]:52
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[27]:52
  Expression: norm(d2ψt - d2ψ[i, :, :]) < 1.0e-8
   Evaluated: 0.19365975159735593 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[27]:52
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[27]:52
  Expression: norm(d2ψt - d2ψ[i, :, :]) < 1.0e-8
   Evaluated: 0.06090280420085852 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[27]:52
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
T

Stacktrace:
 [1] top-level scope at In[27]:52
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[27]:52
  Expression: norm(d2ψt - d2ψ[i, :, :]) < 1.0e-8
   Evaluated: 0.19148718733341147 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[27]:52
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[27]:52
  Expression: norm(d2ψt - d2ψ[i, :, :]) < 1.0e-8
   Evaluated: 0.19988472065455848 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[27]:52
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of ex

Stacktrace:
 [1] top-level scope at In[27]:52
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[27]:52
  Expression: norm(d2ψt - d2ψ[i, :, :]) < 1.0e-8
   Evaluated: 0.017799759861096012 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[27]:52
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[27]:52
  Expression: norm(d2ψt - d2ψ[i, :, :]) < 1.0e-8
   Evaluated: 0.15088943178115694 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[27]:52
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of e

Stacktrace:
 [1] top-level scope at In[27]:52
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[27]:52
  Expression: norm(d2ψt - d2ψ[i, :, :]) < 1.0e-8
   Evaluated: 0.007283675988180931 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[27]:52
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[27]:52
  Expression: norm(d2ψt - d2ψ[i, :, :]) < 1.0e-8
   Evaluated: 0.19312863490306245 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[27]:52
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of e

...

Stacktrace:
 [1] top-level scope at In[27]:69
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[27]:69
  Expression: abs(d2ψxd[i] - d2ψxd_t) < 1.0e-10
   Evaluated: 0.19902414292687115 < 1.0e-10
Stacktrace:
 [1] top-level scope at In[27]:69
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[27]:69
  Expression: abs(d2ψxd[i] - d2ψxd_t) < 1.0e-10
   Evaluated: 0.18849084244734007 < 1.0e-10
Stacktrace:
 [1] top-level scope at In[27]:69
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of ex

Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[27]:69
  Expression: abs(d2ψxd[i] - d2ψxd_t) < 1.0e-10
   Evaluated: 0.12406673157747518 < 1.0e-10
Stacktrace:
 [1] top-level scope at In[27]:69
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[27]:69
  Expression: abs(d2ψxd[i] - d2ψxd_t) < 1.0e-10
   Evaluated: 0.19699153646445672 < 1.0e-10
Stacktrace:
 [1] top-level scope at In[27]:69
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[27]:69
  Expression: abs(d2ψxd[i] - d2ψxd_t) < 1.0e-10
   Evaluated: 0.09895511097880806 < 1.0e-10
Stacktrace:
 [1] top-level scope at In[27]:69
 [2] t

Excessive output truncated after 524312 bytes.


Stacktrace:
 [1] top-level scope at In[27]:69
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[27]:69
  Expression: abs(d2ψxd[i] - d2ψxd_t) < 1.0e-10
   Evaluated: 0.14313930211717646 < 1.0e-10
Stacktrace:
 [1] top-level scope at In[27]:69
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[27]:69
  Expression: abs(d2ψxd[i] - d2ψxd_t) < 1.0e-10
   Evaluated: 0.09491761447448899 < 1.0e-10
Stacktrace:
 [1] top-level scope at In[27]:69
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[27]:3
Test evaluation, gradient and hessian of e

TestSetException: Some tests did not pass: 11 passed, 1506 failed, 0 errored, 0 broken.

In [26]:





@testset "Test evaluation, gradient and hessian of expanded function Nx = 1 II" begin

    Nx = 1
    Ne = 500
    ens = EnsembleState(Nx, Ne)
    ens.S .= randn(Nx, Ne)


    B = MultiBasis(BasisProHermite(3; scaled =true), Nx)

    idx = reshape([0; 1; 2; 3], (4, 1))
    truncidx = idx[1:2:end,:]

    coeff =  randn(size(idx,1))

    f = ExpandedFunction(B, idx, coeff)



    ψt_basis, dψt_basis, d2ψt_basis = alleval(f, ens)

    ψ_basis = evaluate_basis(f, ens.S)
    dψ_basis = grad_x_basis(f, ens.S)
    d2ψ_basis = hess_x_basis(f, ens.S);
    @test norm(ψt_basis - ψ_basis)<1e-8
    @test norm(dψt_basis - dψ_basis)<1e-8
    @test norm(d2ψt_basis - d2ψ_basis)<1e-8


    # For truncated basis
    ψtrunc_basis = evaluate_basis(f, ens.S, truncidx)
    dψtrunc_basis = grad_x_basis(f, ens.S, truncidx)
    d2ψtrunc_basis = hess_x_basis(f, ens.S, truncidx);

    @test norm(ψ_basis[:,1:2:end] - ψtrunc_basis)<1e-8
    @test norm(dψ_basis[:,1:2:end,:] - dψtrunc_basis)<1e-8
    @test norm(d2ψt_basis[:,1:2:end,:,:] - d2ψtrunc_basis)<1e-8


    # Verify function evaluation
    @test norm(map(i->f(member(ens,i)),1:Ne) - evaluate(f, ens.S))<1e-8

    #  Verify gradient
    dψ = grad_x(f, ens.S)
    @test norm(hcat(map(i->ForwardDiff.gradient(f, member(ens,i)), 1:Ne)...)' - dψ)<1e-8

    # Verify hessian
    d2ψ = hess_x(f, ens.S)

    for i=1:Ne
        d2ψt = ForwardDiff.hessian(f, member(ens,i))
        @test norm(d2ψt - d2ψ[i,:,:])<1e-8
    end


    # Verify grad_xd
    dψxd = grad_xd(f, ens.S)

    for i=1:Ne
        dψxd_t = ForwardDiff.gradient(f, member(ens,i))[end]
        @test abs(dψxd[i] - dψxd_t)<1e-10
    end

    # Verify hess_xd
    d2ψxd = hess_xd(f, ens.S)

    for i=1:Ne
        d2ψxd_t = ForwardDiff.hessian(f, member(ens,i))[end,end]
        @test abs(d2ψxd[i] - d2ψxd_t)<1e-10
    end

    # Verify grad_coeff
    dψcoeff  = grad_coeff(f, ens.S)
    dψcoefftrunc  = grad_coeff(f, ens.S, collect(1:2:size(idx,1)))
    @test norm(dψcoeff[:,1:2:end] - dψcoefftrunc)<1e-10

    @test norm(dψcoeff - ψ_basis)<1e-10
    @test norm(dψcoefftrunc - ψtrunc_basis)<1e-10

    # Verify hess_coeff
    d2ψcoeff  = hess_coeff(f, ens.S)
    d2ψcoefftrunc  = hess_coeff(f, ens.S, collect(1:2:size(idx,1)))

    @test norm(d2ψcoeff)<1e-10

    # Verify grad_coeff_grad_xd
    dψcoeffxd = grad_coeff_grad_xd(f, ens.S)
    dψcoeffxdtrunc = grad_coeff_grad_xd(f, ens.S, collect(1:2:size(idx,1)))

    @test norm(dψcoeffxd[:,1:2:end] - dψcoeffxdtrunc)<1e-10
    @test norm(dψcoeffxd - grad_xk_basis(f, ens.S, 1, Nx))<1e-10
    @test norm(dψcoeffxdtrunc - grad_xk_basis(f, ens.S, 1, Nx, f.idx[collect(1:2:size(idx,1)),:]))<1e-10

    # Verify hess_coeff_grad_xd
    d2ψcoeffxd = hess_coeff_grad_xd(f, ens.S)
    d2ψcoeffxdtrunc = hess_coeff_grad_xd(f, ens.S, collect(1:2:size(idx,1)))

    @test norm(d2ψcoeffxd)<1e-10
    @test norm(d2ψcoeffxdtrunc)<1e-10
end

@testset "Test evaluation, gradient and hessian of expanded function Nx = 2" begin

    Nx = 2
    Ne = 500
    ens = EnsembleState(Nx, Ne)
    ens.S .= randn(Nx, Ne)


    B = MultiBasis(BasisProHermite(3; scaled =true), Nx)

    idx = [0 0]
    truncidx = idx[1:2:end,:]

    coeff =  randn(size(idx,1))

    f = ExpandedFunction(B, idx, coeff)



    ψt_basis, dψt_basis, d2ψt_basis = alleval(f, ens)

    ψ_basis = evaluate_basis(f, ens.S)
    dψ_basis = grad_x_basis(f, ens.S)
    d2ψ_basis = hess_x_basis(f, ens.S);
    @test norm(ψt_basis - ψ_basis)<1e-8
    @test norm(dψt_basis - dψ_basis)<1e-8
    @test norm(d2ψt_basis - d2ψ_basis)<1e-8


    # For truncated basis
    ψtrunc_basis = evaluate_basis(f, ens.S, truncidx)
    dψtrunc_basis = grad_x_basis(f, ens.S, truncidx)
    d2ψtrunc_basis = hess_x_basis(f, ens.S, truncidx);

    @test norm(ψ_basis[:,1:2:end] - ψtrunc_basis)<1e-8
    @test norm(dψ_basis[:,1:2:end,:] - dψtrunc_basis)<1e-8
    @test norm(d2ψt_basis[:,1:2:end,:,:] - d2ψtrunc_basis)<1e-8


    # Verify function evaluation
    @test norm(map(i->f(member(ens,i)),1:Ne) - evaluate(f, ens.S))<1e-8

    #  Verify gradient
    dψ = grad_x(f, ens.S)
    @test norm(hcat(map(i->ForwardDiff.gradient(f, member(ens,i)), 1:Ne)...)' - dψ)<1e-8

    # Verify hessian
    d2ψ = hess_x(f, ens.S)

    for i=1:Ne
        d2ψt = ForwardDiff.hessian(f, member(ens,i))
        @test norm(d2ψt - d2ψ[i,:,:])<1e-8
    end


    # Verify grad_xd
    dψxd = grad_xd(f, ens.S)

    for i=1:Ne
        dψxd_t = ForwardDiff.gradient(f, member(ens,i))[end]
        @test abs(dψxd[i] - dψxd_t)<1e-10
    end

    # Verify hess_xd
    d2ψxd = hess_xd(f, ens.S)

    for i=1:Ne
        d2ψxd_t = ForwardDiff.hessian(f, member(ens,i))[end,end]
        @test abs(d2ψxd[i] - d2ψxd_t)<1e-10
    end

    # Verify grad_coeff
    dψcoeff  = grad_coeff(f, ens.S)
    dψcoefftrunc  = grad_coeff(f, ens.S, collect(1:2:size(idx,1)))
    @test norm(dψcoeff[:,1:2:end] - dψcoefftrunc)<1e-10

    @test norm(dψcoeff - ψ_basis)<1e-10
    @test norm(dψcoefftrunc - ψtrunc_basis)<1e-10

    # Verify hess_coeff
    d2ψcoeff  = hess_coeff(f, ens.S)
    d2ψcoefftrunc  = hess_coeff(f, ens.S, collect(1:2:size(idx,1)))

    @test norm(d2ψcoeff)<1e-10

    # Verify grad_coeff_grad_xd
    dψcoeffxd = grad_coeff_grad_xd(f, ens.S)
    dψcoeffxdtrunc = grad_coeff_grad_xd(f, ens.S, collect(1:2:size(idx,1)))

    @test norm(dψcoeffxd[:,1:2:end] - dψcoeffxdtrunc)<1e-10
    @test norm(dψcoeffxd - grad_xk_basis(f, ens.S, 1, Nx))<1e-10
    @test norm(dψcoeffxdtrunc - grad_xk_basis(f, ens.S, 1, Nx, f.idx[collect(1:2:size(idx,1)),:]))<1e-10

    # Verify hess_coeff_grad_xd
    d2ψcoeffxd = hess_coeff_grad_xd(f, ens.S)
    d2ψcoeffxdtrunc = hess_coeff_grad_xd(f, ens.S, collect(1:2:size(idx,1)))

    @test norm(d2ψcoeffxd)<1e-10
    @test norm(d2ψcoeffxdtrunc)<1e-10
end

@testset "Test evaluation, gradient and hessian of expanded function Nx = 2" begin

    Nx = 2
    Ne = 500
    ens = EnsembleState(Nx, Ne)
    ens.S .= randn(Nx, Ne)


    B = MultiBasis(BasisProHermite(3; scaled =true), Nx)

    idx = [0 0; 0 1; 1 0; 1 1; 1 2]
    truncidx = idx[1:2:end,:]

    coeff =  randn(size(idx,1))

    f = ExpandedFunction(B, idx, coeff)



    ψt_basis, dψt_basis, d2ψt_basis = alleval(f, ens)

    ψ_basis = evaluate_basis(f, ens.S)
    dψ_basis = grad_x_basis(f, ens.S)
    d2ψ_basis = hess_x_basis(f, ens.S);
    @test norm(ψt_basis - ψ_basis)<1e-8
    @test norm(dψt_basis - dψ_basis)<1e-8
    @test norm(d2ψt_basis - d2ψ_basis)<1e-8


    # For truncated basis
    ψtrunc_basis = evaluate_basis(f, ens.S, truncidx)
    dψtrunc_basis = grad_x_basis(f, ens.S, truncidx)
    d2ψtrunc_basis = hess_x_basis(f, ens.S, truncidx);

    @test norm(ψ_basis[:,1:2:end] - ψtrunc_basis)<1e-8
    @test norm(dψ_basis[:,1:2:end,:] - dψtrunc_basis)<1e-8
    @test norm(d2ψt_basis[:,1:2:end,:,:] - d2ψtrunc_basis)<1e-8


    # Verify function evaluation
    @test norm(map(i->f(member(ens,i)),1:Ne) - evaluate(f, ens.S))<1e-8

    #  Verify gradient
    dψ = grad_x(f, ens.S)
    @test norm(hcat(map(i->ForwardDiff.gradient(f, member(ens,i)), 1:Ne)...)' - dψ)<1e-8

    # Verify hessian
    d2ψ = hess_x(f, ens.S)

    for i=1:Ne
        d2ψt = ForwardDiff.hessian(f, member(ens,i))
        @test norm(d2ψt - d2ψ[i,:,:])<1e-8
    end


    # Verify grad_xd
    dψxd = grad_xd(f, ens.S)

    for i=1:Ne
        dψxd_t = ForwardDiff.gradient(f, member(ens,i))[end]
        @test abs(dψxd[i] - dψxd_t)<1e-10
    end

    # Verify hess_xd
    d2ψxd = hess_xd(f, ens.S)

    for i=1:Ne
        d2ψxd_t = ForwardDiff.hessian(f, member(ens,i))[end,end]
        @test abs(d2ψxd[i] - d2ψxd_t)<1e-10
    end

    # Verify grad_coeff
    dψcoeff  = grad_coeff(f, ens.S)
    dψcoefftrunc  = grad_coeff(f, ens.S, collect(1:2:size(idx,1)))
    @test norm(dψcoeff[:,1:2:end] - dψcoefftrunc)<1e-10

    @test norm(dψcoeff - ψ_basis)<1e-10
    @test norm(dψcoefftrunc - ψtrunc_basis)<1e-10

    # Verify hess_coeff
    d2ψcoeff  = hess_coeff(f, ens.S)
    d2ψcoefftrunc  = hess_coeff(f, ens.S, collect(1:2:size(idx,1)))

    @test norm(d2ψcoeff)<1e-10

    # Verify grad_coeff_grad_xd
    dψcoeffxd = grad_coeff_grad_xd(f, ens.S)
    dψcoeffxdtrunc = grad_coeff_grad_xd(f, ens.S, collect(1:2:size(idx,1)))

    @test norm(dψcoeffxd[:,1:2:end] - dψcoeffxdtrunc)<1e-10
    @test norm(dψcoeffxd - grad_xk_basis(f, ens.S, 1, Nx))<1e-10
    @test norm(dψcoeffxdtrunc - grad_xk_basis(f, ens.S, 1, Nx, f.idx[collect(1:2:size(idx,1)),:]))<1e-10

    # Verify hess_coeff_grad_xd
    d2ψcoeffxd = hess_coeff_grad_xd(f, ens.S)
    d2ψcoeffxdtrunc = hess_coeff_grad_xd(f, ens.S, collect(1:2:size(idx,1)))

    @test norm(d2ψcoeffxd)<1e-10
    @test norm(d2ψcoeffxdtrunc)<1e-10
end

@testset "Test evaluation, gradient and hessian of expanded function Nx = 3 I" begin

    Nx = 3
    Ne = 500
    ens = EnsembleState(Nx, Ne)
    ens.S .= randn(Nx, Ne)

    B = MultiBasis(BasisProHermite(3; scaled =true), Nx)

    idx = [0 0 0; 0 0 1; 0 1 0; 0 1 1; 0 1 2; 1 0 0]
    truncidx = idx[1:2:end,:]
    coeff =  randn(size(idx,1))

    f = ExpandedFunction(B, idx, coeff)

    ψt_basis, dψt_basis, d2ψt_basis = alleval(f, ens)

    ψ_basis = evaluate_basis(f, ens.S)
    dψ_basis = grad_x_basis(f, ens.S)
    d2ψ_basis = hess_x_basis(f, ens.S);
    @test norm(ψt_basis - ψ_basis)<1e-8
    @test norm(dψt_basis - dψ_basis)<1e-8
    @test norm(d2ψt_basis - d2ψ_basis)<1e-8


    # For truncated basis
    ψtrunc_basis = evaluate_basis(f, ens.S, truncidx)
    dψtrunc_basis = grad_x_basis(f, ens.S, truncidx)
    d2ψtrunc_basis = hess_x_basis(f, ens.S, truncidx);

    @test norm(ψ_basis[:,1:2:end] - ψtrunc_basis)<1e-8
    @test norm(dψ_basis[:,1:2:end,:] - dψtrunc_basis)<1e-8
    @test norm(d2ψt_basis[:,1:2:end,:,:] - d2ψtrunc_basis)<1e-8


    # Verify function evaluation
    @test norm(map(i->f(member(ens,i)),1:Ne) - evaluate(f, ens.S))<1e-8

    #  Verify gradient
    dψ = grad_x(f, ens.S)
    @test norm(hcat(map(i->ForwardDiff.gradient(f, member(ens,i)), 1:Ne)...)' - dψ)<1e-8

    # Verify hessian
    d2ψ = hess_x(f, ens.S)

    for i=1:Ne
        d2ψt = ForwardDiff.hessian(f, member(ens,i))
        @test norm(d2ψt - d2ψ[i,:,:])<1e-8
    end


    # Verify grad_xd
    dψxd = grad_xd(f, ens.S)

    for i=1:Ne
        dψxd_t = ForwardDiff.gradient(f, member(ens,i))[end]
        @test abs(dψxd[i] - dψxd_t)<1e-10
    end

    # Verify hess_xd
    d2ψxd = hess_xd(f, ens.S)

    for i=1:Ne
        d2ψxd_t = ForwardDiff.hessian(f, member(ens,i))[end,end]
        @test abs(d2ψxd[i] - d2ψxd_t)<1e-10
    end

    # Verify grad_coeff
    dψcoeff  = grad_coeff(f, ens.S)
    dψcoefftrunc  = grad_coeff(f, ens.S, collect(1:2:size(idx,1)))
    @test norm(dψcoeff[:,1:2:end] - dψcoefftrunc)<1e-10

    @test norm(dψcoeff - ψ_basis)<1e-10
    @test norm(dψcoefftrunc - ψtrunc_basis)<1e-10

    # Verify hess_coeff
    d2ψcoeff  = hess_coeff(f, ens.S)
    d2ψcoefftrunc  = hess_coeff(f, ens.S, collect(1:2:size(idx,1)))

    @test norm(d2ψcoeff)<1e-10

    # Verify grad_coeff_grad_xd
    dψcoeffxd = grad_coeff_grad_xd(f, ens.S)
    dψcoeffxdtrunc = grad_coeff_grad_xd(f, ens.S, collect(1:2:size(idx,1)))

    @test norm(dψcoeffxd[:,1:2:end] - dψcoeffxdtrunc)<1e-10
    @test norm(dψcoeffxd - grad_xk_basis(f, ens.S, 1, Nx))<1e-10
    @test norm(dψcoeffxdtrunc - grad_xk_basis(f, ens.S, 1, Nx, f.idx[collect(1:2:size(idx,1)),:]))<1e-10

    # Verify hess_coeff_grad_xd
    d2ψcoeffxd = hess_coeff_grad_xd(f, ens.S)
    d2ψcoeffxdtrunc = hess_coeff_grad_xd(f, ens.S, collect(1:2:size(idx,1)))

    @test norm(d2ψcoeffxd)<1e-10
    @test norm(d2ψcoeffxdtrunc)<1e-10
end

@testset "Test evaluation, gradient and hessian of expanded function Nx = 3 II" begin

    Nx = 3
    Ne = 500
    ens = EnsembleState(Nx, Ne)
    ens.S .= randn(Nx, Ne)

    B = MultiBasis(BasisProHermite(3; scaled =true), Nx)

    idx = [0 0 0]
    truncidx = idx[1:2:end,:]
    coeff =  randn(size(idx,1))

    f = ExpandedFunction(B, idx, coeff)

    ψt_basis, dψt_basis, d2ψt_basis = alleval(f, ens)

    ψ_basis = evaluate_basis(f, ens.S)
    dψ_basis = grad_x_basis(f, ens.S)
    d2ψ_basis = hess_x_basis(f, ens.S);
    @test norm(ψt_basis - ψ_basis)<1e-8
    @test norm(dψt_basis - dψ_basis)<1e-8
    @test norm(d2ψt_basis - d2ψ_basis)<1e-8


    # For truncated basis
    ψtrunc_basis = evaluate_basis(f, ens.S, truncidx)
    dψtrunc_basis = grad_x_basis(f, ens.S, truncidx)
    d2ψtrunc_basis = hess_x_basis(f, ens.S, truncidx);

    @test norm(ψ_basis[:,1:2:end] - ψtrunc_basis)<1e-8
    @test norm(dψ_basis[:,1:2:end,:] - dψtrunc_basis)<1e-8
    @test norm(d2ψt_basis[:,1:2:end,:,:] - d2ψtrunc_basis)<1e-8


    # Verify function evaluation
    @test norm(map(i->f(member(ens,i)),1:Ne) - evaluate(f, ens.S))<1e-8

    #  Verify gradient
    dψ = grad_x(f, ens.S)
    @test norm(hcat(map(i->ForwardDiff.gradient(f, member(ens,i)), 1:Ne)...)' - dψ)<1e-8

    # Verify hessian
    d2ψ = hess_x(f, ens.S)

    for i=1:Ne
        d2ψt = ForwardDiff.hessian(f, member(ens,i))
        @test norm(d2ψt - d2ψ[i,:,:])<1e-8
    end


    # Verify grad_xd
    dψxd = grad_xd(f, ens.S)

    for i=1:Ne
        dψxd_t = ForwardDiff.gradient(f, member(ens,i))[end]
        @test abs(dψxd[i] - dψxd_t)<1e-10
    end

    # Verify hess_xd
    d2ψxd = hess_xd(f, ens.S)

    for i=1:Ne
        d2ψxd_t = ForwardDiff.hessian(f, member(ens,i))[end,end]
        @test abs(d2ψxd[i] - d2ψxd_t)<1e-10
    end

    # Verify grad_coeff
    dψcoeff  = grad_coeff(f, ens.S)
    dψcoefftrunc  = grad_coeff(f, ens.S, collect(1:2:size(idx,1)))
    @test norm(dψcoeff[:,1:2:end] - dψcoefftrunc)<1e-10

    @test norm(dψcoeff - ψ_basis)<1e-10
    @test norm(dψcoefftrunc - ψtrunc_basis)<1e-10

    # Verify hess_coeff
    d2ψcoeff  = hess_coeff(f, ens.S)
    d2ψcoefftrunc  = hess_coeff(f, ens.S, collect(1:2:size(idx,1)))

    @test norm(d2ψcoeff)<1e-10

    # Verify grad_coeff_grad_xd
    dψcoeffxd = grad_coeff_grad_xd(f, ens.S)
    dψcoeffxdtrunc = grad_coeff_grad_xd(f, ens.S, collect(1:2:size(idx,1)))

    @test norm(dψcoeffxd[:,1:2:end] - dψcoeffxdtrunc)<1e-10
    @test norm(dψcoeffxd - grad_xk_basis(f, ens.S, 1, Nx))<1e-10
    @test norm(dψcoeffxdtrunc - grad_xk_basis(f, ens.S, 1, Nx, f.idx[collect(1:2:size(idx,1)),:]))<1e-10

    # Verify hess_coeff_grad_xd
    d2ψcoeffxd = hess_coeff_grad_xd(f, ens.S)
    d2ψcoeffxdtrunc = hess_coeff_grad_xd(f, ens.S, collect(1:2:size(idx,1)))

    @test norm(d2ψcoeffxd)<1e-10
    @test norm(d2ψcoeffxdtrunc)<1e-10
end

Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[26]:28
  Expression: norm(ψt_basis - ψ_basis) < 1.0e-8
   Evaluated: 11.306324020932555 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[26]:28
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[26]:29
  Expression: norm(dψt_basis - dψ_basis) < 1.0e-8
   Evaluated: 4.2122654069914205 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[26]:29
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[26]:30
  Expression: norm(d2ψt_basis - d2ψ_basis) < 1.0e-8
   Evaluated: 4.874835725340076 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[26]:30
 [2] to

   Evaluated: 0.1861228117853838 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[26]:55
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[26]:55
  Expression: norm(d2ψt - d2ψ[i, :, :]) < 1.0e-8
   Evaluated: 0.19365975159735593 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[26]:55
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[26]:55
  Expression: norm(d2ψt - d2ψ[i, :, :]) < 1.0e-8
   Evaluated: 0.06090280420085852 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[26]:55
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
T

   Evaluated: 0.19148718733341147 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[26]:55
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[26]:55
  Expression: norm(d2ψt - d2ψ[i, :, :]) < 1.0e-8
   Evaluated: 0.19988472065455848 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[26]:55
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[26]:55
  Expression: norm(d2ψt - d2ψ[i, :, :]) < 1.0e-8
   Evaluated: 0.19939125661288143 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[26]:55
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6


   Evaluated: 0.017799759861096012 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[26]:55
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[26]:55
  Expression: norm(d2ψt - d2ψ[i, :, :]) < 1.0e-8
   Evaluated: 0.15088943178115694 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[26]:55
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[26]:55
  Expression: norm(d2ψt - d2ψ[i, :, :]) < 1.0e-8
   Evaluated: 0.1961357497857296 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[26]:55
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6


Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[26]:55
  Expression: norm(d2ψt - d2ψ[i, :, :]) < 1.0e-8
   Evaluated: 0.19312863490306245 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[26]:55
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[26]:55
  Expression: norm(d2ψt - d2ψ[i, :, :]) < 1.0e-8
   Evaluated: 0.13082405952292217 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[26]:55
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[26]:55
  Expression: norm(d2ψt - d2ψ[i, :, :]) < 1.0e-8
   Evaluated: 0.17200836384900825 < 1.0e-8
Stacktrace:
 [1] top-level scope at In[26]:55
 [2] t

...

   Evaluated: 0.18706016734440578 < 1.0e-10
Stacktrace:
 [1] top-level scope at In[26]:72
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[26]:72
  Expression: abs(d2ψxd[i] - d2ψxd_t) < 1.0e-10
   Evaluated: 0.10925672187473048 < 1.0e-10
Stacktrace:
 [1] top-level scope at In[26]:72
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[26]:72
  Expression: abs(d2ψxd[i] - d2ψxd_t) < 1.0e-10
   Evaluated: 0.1740916118732232 < 1.0e-10
Stacktrace:
 [1] top-level scope at In[26]:72
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6


Stacktrace:
 [1] top-level scope at In[26]:72
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[26]:72
  Expression: abs(d2ψxd[i] - d2ψxd_t) < 1.0e-10
   Evaluated: 0.16035073280289858 < 1.0e-10
Stacktrace:
 [1] top-level scope at In[26]:72
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[26]:72
  Expression: abs(d2ψxd[i] - d2ψxd_t) < 1.0e-10
   Evaluated: 0.19912703622425612 < 1.0e-10
Stacktrace:
 [1] top-level scope at In[26]:72
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
Test evaluation, gradient and hessian of ex

Excessive output truncated after 524312 bytes.


 [1] top-level scope at In[26]:72
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[26]:72
  Expression: abs(d2ψxd[i] - d2ψxd_t) < 1.0e-10
   Evaluated: 0.07455992278549957 < 1.0e-10
Stacktrace:
 [1] top-level scope at In[26]:72
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
Test evaluation, gradient and hessian of expanded function Nx = 1 I: Test Failed at In[26]:72
  Expression: abs(d2ψxd[i] - d2ψxd_t) < 1.0e-10
   Evaluated: 0.14313930211717646 < 1.0e-10
Stacktrace:
 [1] top-level scope at In[26]:72
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113
 [3] top-level scope at In[26]:6
Test evaluation, gradient and hessian of expanded func

TestSetException: Some tests did not pass: 11 passed, 1506 failed, 0 errored, 0 broken.

### Greedy fit

In [9]:
k = 2
Ne = 8
m = 5

X  =  Matrix([0.267333   1.43021;
          0.364979   0.607224;
         -1.23693    0.249277;
         -2.0526     0.915629;
         -0.182465   0.415874;
          0.412907   1.01672;
          1.41332   -0.918205;
          0.766647  -1.00445]');

H = HermiteMapk(m, k; α = 0.0)
S = Storage(H.I.f, X);

J = 0.0
dJ = zeros(1)
@time negative_log_likelihood!(J, dJ, getcoeff(H), S, H, X);

  0.000034 seconds (264 allocations: 23.109 KiB)


In [6]:
@sJ, dJ

(J, dJ) = (1.3172777885451104, [0.33903487500000007])


(1.3172777885451104, [0.33903487500000007])

In [10]:
Nx = 2
Ne = 8
m = 5
ens = EnsembleState(Nx, Ne)

ens.S .=  [0.267333   1.43021;
          0.364979   0.607224;
         -1.23693    0.249277;
         -2.0526     0.915629;
         -0.182465   0.415874;
          0.412907   1.01672;
          1.41332   -0.918205;
          0.766647  -1.00445]';
X = ens.S

coeff = [0.0];

# coeff = [0.6285037650645056;
#           -0.4744029092496623;
#            1.1405280011620331;
#           -0.7217760771894809;
#            0.11855056306742319];
# Initialize map with zero coefficients
# Hk_old = HermiteMapk(m, Nx, [0 0; 0 1; 0 2; 1 0; 2 0], coeff; α = 1e-2);
Hk_old = HermiteMapk(m, Nx, [0 0], coeff; α = 1e-2);
S = Storage(Hk_old.I.f, X)

# reduced_margin = getreducedmargin(getidx(Hk_old))
# @show reduced_margin
# @time idx_new, reduced_margin = update_component(Hk_old, X, reduced_margin, S)

# @show idx_new
# @show reduced_margin

J = 0.0
dJ = zeros(1)
J = negative_log_likelihood!(J, dJ, getcoeff(Hk_old), S, Hk_old, X)

@show J, dJ
# idx_new = vcat(getidx(Hk_old), reduced_margin)
# Hk_new = HermiteMapk(m, Nx, idx_new, vcat(getcoeff(Hk_old), zeros(size(reduced_margin,1))); α = 1.0);
# # coeff_new, coeff_idx_added, idx_added = update_coeffs(Hk_old, Hk_new)

(J, dJ) = (1.3172777885451104, [0.33903487500000007])


In [13]:
res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S, Hk_old, ens.S)), coeff, Optim.BFGS())
coeffopt = Optim.minimizer(res)

1-element Array{Float64,1}:
 -0.33238713235294115